In [1]:
from olfactorybulb.database import *
import os,sys
from neuronunit.tests.olfactory_bulb.publications import *
from neuronunit.tests.olfactory_bulb.tests import *
from neuronunit.models.neuron_cell import NeuronCellModel
from sciunit.suites import TestSuite
from pandas import DataFrame
import quantities as pq
from neuronunit.tests.olfactory_bulb.utilities import cache
from linetimer import CodeTimer
import string, math
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
#import smt
#from smt.surrogate_models.genn import GENN, load_smt_data
from scipy import optimize
import linetimer
import multiprocessing
from multiprocessing import Pool, TimeoutError
from sciunit.scores import ZScore

/home/justas/anaconda2/envs/p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Could not load NEURONBackend


In [2]:
cell_type = 'mc'

# Load tests
measurements = Measurement\
    .select(Measurement)\
    .join(Property)\
    .switch(Measurement)\
    .join(Source)\
    .where((Measurement.property.type == "Electrophysiology") & (Measurement.property.id.startswith(cell_type+'_')))\
    .order_by(Measurement.property.id)

properties = {}

for m in measurements:
    test_generic = str(m.property.test_class_generic)
    pub = str(m.source.publication_class).strip()
    class_name = test_generic+pub
    
    if test_generic not in properties:
        properties[test_generic] = []
    
    globals()[class_name] = type(class_name, 
                                 (eval(pub), eval(test_generic)), 
                                 {})
    
    print('creating specific test class: ' + class_name)
    
    test_instance = eval(class_name)(observation={"mean": m.mean * eval(m.property.units), "std": m.std * eval(m.property.units), "n": m.n})
    
    properties[test_generic].append(test_instance)
    
    
# Load model classes
model_classes = list(CellModel\
                    .select(CellModel)\
                    .where(CellModel.cell_type == cell_type.upper())             
                     )
for i, m in enumerate(model_classes):
    nmsp = string.join(m.isolated_model_class.split('.')[:-1],'.')
    cls = m.isolated_model_class.split('.')[-1]

    import_cmd = 'from '+nmsp+' import '+cls+' as Model'+str(i)
    print(import_cmd)
    exec(import_cmd)
    
# Create work item list
work_items = []

for model in model_classes:
    work_items.append({"model_class": model.isolated_model_class })
        
def dowork(item):
    results = item
    results["properties"] = {}
    results["model_score"] = 0
    
    import prev_ob_models
    exec('cell = '+ str(item["model_class"]) +'()')

    model = NeuronCellModel(cell.soma(0.5),name=cell.__class__.__module__+'.'+cell.__class__.__name__)


    for prop in properties.keys():
        
        if prop not in results["properties"]:
            results["properties"][prop] = { "tests": {}, "total_n": 0, "z_score_combined": None}
        
        prop_tests = properties[prop]
        
        for prop_test in prop_tests:
            
            prop_test_result = {}
            results["properties"][prop]["tests"][prop_test.__class__.__name__] = prop_test_result          
            
            try:
                #print('Starting', item, prop_test)
                prediction = prop_test.generate_prediction(model)
                #print('Finished', item, prop_test, prediction)

            except:
                import traceback
                prediction = traceback.format_exc()
                print(prediction)
                
            prop_test_result["observation"] = prop_test.observation
            prop_test_result["prediction"] = prediction
            
            if type(prediction) != str:
                z_score = (prediction - prop_test.observation["mean"])/prop_test.observation["std"]
                z_score = z_score.simplified
            else:
                z_score = 6.0 # errors are treated as 6 std deviation
                
            z_weighed = z_score * prop_test.observation["n"]

            prop_test_result["z_score"] = z_score
            prop_test_result["z_score_weighed"] = z_weighed
            
            results["properties"][prop]["total_n"] += prop_test.observation["n"]
            
        results["properties"][prop]["z_score_combined"] = sum([i["z_score_weighed"] for i in results["properties"][prop]["tests"].values()])
        results["properties"][prop]["z_score_combined"] /= results["properties"][prop]["total_n"]
        
        results["model_score"] += results["properties"][prop]["z_score_combined"].magnitude**2
        
    import math
    results["model_score"] = math.sqrt(results["model_score"])

    return results


creating specific test class: AfterHyperpolarizationAmplitudeTestYu2015
creating specific test class: AfterHyperpolarizationAmplitudeTestBurtonUrban2014
creating specific test class: AfterHyperpolarizationTimeTestYu2015
creating specific test class: AfterHyperpolarizationTimeTestBurtonUrban2014
creating specific test class: SpikeAmplitudeTestYu2015
creating specific test class: SpikeAmplitudeTestBurtonUrban2014
creating specific test class: SpikeHalfWidthTestYu2015
creating specific test class: SpikeHalfWidthTestBurtonUrban2014
creating specific test class: SpikePeakTestYu2015
creating specific test class: SpikeThresholdTestYu2015
creating specific test class: SpikeThresholdTestBurtonUrban2014
creating specific test class: CellCapacitanceTestYu2015
creating specific test class: CellCapacitanceTestBurtonUrban2014
creating specific test class: FISlopeTestBurtonUrban2014
creating specific test class: InputResistanceTestYu2015
creating specific test class: InputResistanceTestBurtonUrban201

In [3]:
#cache.clear()

In [4]:
# Create helper classes to speed up fitting

class BurtonUrban2014(BurtonUrban2014):
    current = 0 * pq.nA
    spike_train_method = "constant_current"
    spike_train_current = 0.111 * pq.nA + 0.0969 * pq.nA
    rheobase = 0.111 * pq.nA
    
class RestingSpikesTest(BurtonUrban2014, SpikesAtCurrentTest):
    pass
    
class SpikesAt20HzCurrentTest(BurtonUrban2014, SpikesAtConstantTrainCurrentTest):
    pass
        
class RheobaseSpikesTest(BurtonUrban2014, RheobaseSpikesTestHelper):
    pass

restingSpikesTest = RestingSpikesTest(observation={"mean":0*pq.dimensionless, "std": 1*pq.dimensionless, "n":1})
rheobaseSpikesTest = RheobaseSpikesTest(observation={"mean":1*pq.dimensionless, "std": 0.25*pq.dimensionless, "n":1})
hz20Spikes = SpikesAt20HzCurrentTest(observation={"mean":20*2*pq.dimensionless, "std": 1*pq.dimensionless, "n":1})

for test_name in properties.keys():
    for test_instance in properties[test_name]:
        test_instance.spike_train_method = "constant_current"
        test_instance.spike_train_current = 0.111 * pq.nA + 0.0969 * pq.nA
        #test_instance.rheobase = 0.111 * pq.nA
        test_instance.rheobase = 0.111 * pq.nA


model = Model23()
model.soma

numodel = NeuronCellModel(model.soma(0.5))
numodel

prop_tests = [
    restingSpikesTest,
    rheobaseSpikesTest,
    hz20Spikes,
    properties["InputResistanceTest"][1],
    properties["MembraneTimeConstantTest"][1],
    properties["RestingVoltageTest"][1],
    properties["ReboundSpikingTest"][0],
    properties["CellCapacitanceTest"][1],
    properties["SagVoltageTest"][1],
    properties["SpikeAccommodationTest"][0],
    properties["SpikeAccommodationTimeConstantTest"][0],
    properties["SpikeThresholdTest"][1],
    properties["AfterHyperpolarizationAmplitudeTest"][1],
    properties["AfterHyperpolarizationTimeTest"][1],
    properties["SpikeAmplitudeTest"][1],
    properties["SpikeHalfWidthTest"][1],
    properties["ISICVTest"][1],
    properties["SpikeAccommodationTest"][0],
    properties["SpikeAccommodationTimeConstantTest"][0],
]

for test in prop_tests:
    print('running', test)
    print(test.judge(numodel))


In [34]:
params = [    
    { "start": 34.77,  "attr": "Ra",        "low": 5.0, "high": 100.0, "lists": ["all"] },
    { "start": 2.706,   "attr": "cm",        "low": 0.1, "high": 4.0, "lists": ["all"] },
    { "start": 49.95,  "attr": "ena",       "low": 40.0, "high": 50.0, "lists": ["all"] },
    { "start": -70.03,  "attr": "ek",        "low": -100.0, "high": -70.0, "lists": ["all"] },
    { "start": -64.42,  "attr": "e_pas",     "low": -70.0, "high": -50.0, "lists": ["all"] },
    { "start": 0.0005955, "attr": "g_pas",     "low": 0, "high": 0.00003, "lists": ["all"] },    
    { "start": 0.5955,  "attr": "sh_Na",     "low": 0, "high": 10, "lists": ["all"] },    
    { "start": 10,  "attr": "tau_CaPool",     "low": 1, "high": 500, "lists": ["all"] },    

    { "start":  0.87485,  "attr": "gbar_Na",     "low": 0, "high": 0.05, "lists": ["all"] },
    { "start": 0.0297,  "attr": "gbar_Kd",     "low": 0, "high": 0.04, "lists": ["all"] },
    { "start": 0.000264,  "attr": "gbar_Kslow",  "low": 0, "high": 0.004, "lists": ["all"] },
    { "start": 0.07215,  "attr": "gbar_KA",     "low": 0, "high": 0.005, "lists": ["all"] },
    { "start": 0.001,  "attr": "gbar_KCa",    "low": 0, "high": 0.004, "lists": ["all"] },
    { "start": 0.00081441,  "attr": "gbar_LCa",  "low": 0, "high": 0.001, "lists": ["all"] },
    
    { "start": -30.805,  "attr": "eh",       "low": -40.0, "high": -25.0, "lists": ["apical"] },
    { "start": 0.00335,  "attr": "gbar_Ih",    "low": 0, "high": 0.00003, "lists": ["apical"] },
    { "start": 0.000107,  "attr": "gbar_CaT",    "low": 0, "high": 18e-3, "lists": ["apical"] },
]

In [18]:
def evaluate(ind, raw_scores=False):  
    def do_work():
        from prev_ob_models.Birgiolas2020.isolated_cells import MC
        mc = MC()
        from neuron import h
        
        soma = mc.cell.soma

        model = NeuronCellModel(soma(0.5))

        model.name = "Birgiolas2020MC1" + str(ind)

        for pi, pv in enumerate(ind):
            if params[pi]["attr"] == "tau_CaPool":
                setattr(h, params[pi]["attr"], pv)
            else:
                for param_list in params[pi]["lists"]:
                    for sec in getattr(mc.cell, param_list):
                        setattr(sec, params[pi]["attr"], pv)

        ct = linetimer.CodeTimer(silent=True)

        prop_tests = [
            restingSpikesTest,
            rheobaseSpikesTest,
            hz20Spikes,
            properties["InputResistanceTest"][1],
            properties["MembraneTimeConstantTest"][1],
            properties["RestingVoltageTest"][1],
            properties["ReboundSpikingTest"][0],
            properties["CellCapacitanceTest"][1],
            properties["SagVoltageTest"][1],
            properties["SpikeAccommodationTest"][0],
            properties["SpikeAccommodationTimeConstantTest"][0],
            properties["SpikeThresholdTest"][1],
            properties["AfterHyperpolarizationAmplitudeTest"][1],
            properties["AfterHyperpolarizationTimeTest"][1],
            properties["SpikeAmplitudeTest"][1],
            properties["SpikeHalfWidthTest"][1],
            properties["ISICVTest"][1],
            properties["SpikeAccommodationTest"][0],
            properties["SpikeAccommodationTimeConstantTest"][0],
        ]
        
        
        with ct:
            if raw_scores:
                scores = [test.generate_prediction(model) for test in prop_tests]
                return scores
            else:
                scores = []
                for test in prop_tests:
                    try:
                        scores.append(test.judge(model))
                    except:
                        scores.append(ZScore(6.0))

        run_time = ct.took / 1000.0 / 10.0

        scores = np.array([s.score for s in scores])

        print("Raw Z:",scores,"RT:", ct.took / 1000.0)
        
        return np.sqrt(np.sum(scores**2))+run_time, 
    
    from multiprocess import Pool, TimeoutError
    pool = Pool(processes = 1)
    
    try:
        result = pool.apply_async(do_work).get(timeout=60)
    except TimeoutError:
        print('Simulation timed out')
        pool.terminate()
        result = 9*10,
    except:
        print('Error in simulation')
        import traceback
        print(traceback.format_exc())
        pool.terminate()
        result = 9*10.999,
        
    return result

In [7]:
#eps = np.sqrt(np.finfo(float).eps)

def evaluate_for_opt(individual):
    return evaluate(individual)[0]
    
def grad_evaluate(individual):
    result = optimize.approx_fprime(individual, evaluate_for_opt, np.abs(np.array(individual)*0.0001))
    
    if np.any(np.abs(result)>1e5):
        print('Discarding invalid gradient', result)
        return [None]*len(individual)
    
    return result

In [8]:
class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)

# We sub-class multiprocessing.pool.Pool instead of multiprocessing.Pool
# because the latter is only a wrapper function, not a proper class.
class MyPool(multiprocessing.pool.Pool):
    Process = NoDaemonProcess
    
def get_true_points(n):
    dims = len(params)
    
    if type(n) == int:
        print('getting random points')
        xy = np.random.rand(n, dims)

        for pi, p in enumerate(params):
            lower_bound = p["low"]
            upper_bound = p["high"]

            xy[:,pi] = xy[:,pi]*(upper_bound-lower_bound)+lower_bound
    
    else:
        print('evaluating passed-in points')
        xy = n
    
    processes = 15   
    pool = MyPool(processes = processes, maxtasksperchild=1)    
    z = np.array(pool.map(evaluate_for_opt, xy))
    #grad_z = np.array(pool.map(grad_evaluate, xy))
    grad_z = xy
    pool.terminate()
    pool.join()
    
#     valid_z = np.where(z < 10)
#     xy = xy[valid_z]
#     z = z[valid_z]
#     grad_z = grad_z[valid_z]
    
#     valid_grad = grad_z[:,0] != np.array(None)
#     xy = xy[valid_grad]
#     z = z[valid_grad]
#     grad_z = grad_z[valid_grad]
    
    
    return xy, z, grad_z

In [ ]:
# Training
xt, yt, dyt_dxt = get_true_points(150*len(params))

#px, py, pg = get_true_points(np.array([[ 7.09304053e-01,  4.98941113e+01, -7.00000000e+01,  1.65979225e-01,
#        3.30619495e-02,  3.83766954e-05, -7.00000000e+01]]))

# xt = np.concatenate((xt,px))
# yt = np.concatenate((yt,py))
# dyt_dxt = np.concatenate((dyt_dxt,pg))

print('best initial fitness',np.min(yt))
DataFrame(xt,yt)

In [ ]:
def plot3d(x,y,z):
    from matplotlib.mlab import griddata

    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)
    zi = griddata(x, y, z, xi, yi, interp='linear')

    #Plot the contour mapping and edit the parameter setting according to your data (http://matplotlib.org/api/pyplot_api.html?highlight=contourf#matplotlib.pyplot.contourf)
    CS = plt.contourf(xi, yi, zi)#, 5, levels=[0,50,100,1000],colors=['b','y','r'],vmax=abs(zi).max(), vmin=-abs(zi).max())
    plt.colorbar()
    plt.show()

In [ ]:
def retrain_surrogate():
    try:
        plot3d(xt[:,0],xt[:,1],yt*-1)
    except:
        pass
    
    # GENN
    genn = GENN()
    genn.options["alpha"] = 0.1             # learning rate that controls optimizer step size
    genn.options["beta1"] = 0.9             # tuning parameter to control ADAM optimization
    genn.options["beta2"] = 0.99            # tuning parameter to control ADAM optimization
    genn.options["lambd"] = 0.1             # lambd = 0. = no regularization, lambd > 0 = regularization
    genn.options["gamma"] = 0.0             # gamma = 0. = no grad-enhancement, gamma > 0 = grad-enhancement
    genn.options["deep"] = 2                # number of hidden layers
    genn.options["wide"] = 10                # number of nodes per hidden layer
    genn.options["mini_batch_size"] = 2000    # used to divide data into training batches (use for large data sets)
    genn.options["num_epochs"] = 10         # number of passes through data
    genn.options["num_iterations"] = 50    # number of optimizer iterations per mini-batch
    genn.options["is_print"] = True         # print output (or not)

    load_smt_data(genn, xt, yt, dyt_dxt)    # convenience function to read in data that is in SMT format
    genn.train()                            # API function to train model
    genn.plot_training_history()            # non-API function to plot training history (to check convergence)
    y_pred = genn.predict_values(xt)         # API function to predict values at new (unseen) points
    
    try:
        plot3d(xt[:,0],xt[:,1],yt*-1)
        plot3d(xt[:,0],xt[:,1],y_pred.flatten()*-1)
    except:
        pass
    
    return genn, y_pred

# genn, y_pred = retrain_surrogate()

# plt.plot(yt, y_pred,'o')
# plt.plot([0,100],[0,100],'-')
# plt.xlim((np.min(yt), np.max(yt)))
# plt.ylim((np.min(y_pred), np.max(y_pred)))
# plt.show()

In [35]:
def GA(suggested_pop=None, n=30, NGEN=30):
    #genn.options['print_global'] = False

    from deap import base, creator
    import math

    creator.create("FitnessMin", base.Fitness, weights=(-1,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    import random
    from deap import tools

    def random_indiv():
        result = [random.random()] * len(params)
        for i, pv in enumerate(result):
            result[i] = (params[i]["high"]-params[i]["low"])*pv+params[i]["low"]
        
        return creator.Individual(result)

    toolbox = base.Toolbox()
    toolbox.register("individual", random_indiv)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    def evaluate_surrogate(individual):
        return genn.predict_values(np.array([individual])).flatten()[0],

    lows = [p["low"] for p in params]
    highs = [p["high"] for p in params]
    
    toolbox.register("mate", tools.cxSimulatedBinaryBounded, eta=0.1, low=lows, up=highs)
    toolbox.register("mutate", tools.mutPolynomialBounded, eta=0.1, low=lows, up=highs, indpb=0.1)
    toolbox.register("evaluate", evaluate)



    toolbox.register("select", tools.selNSGA2, k=int(n*0.2))
    if suggested_pop is None:
        pop = toolbox.population(n=n)
    else:
        pop = [creator.Individual(i) for i in suggested_pop]
        
    CXPB, MUTPB = 1, 1
    F_DIVERSITY = 0.5

    # Evaluate the entire population
    processes = 15   
    pool = MyPool(processes = processes, maxtasksperchild=1)    
    fitnesses = pool.map(toolbox.evaluate, pop)
    pool.terminate()
    pool.join()
    
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the parents
        elite = toolbox.select(pop)      

        random_offspring = toolbox.population(n=int(n*F_DIVERSITY/2.0))
        diversity_offspring = random_offspring + tools.selRandom(pop, int(n*F_DIVERSITY/2.0))        
        elite_offspring = tools.selRandom(elite, n-len(elite)-len(diversity_offspring))

        offspring = random_offspring + diversity_offspring + elite_offspring

        # Clone the selected individuals
        offspring = map(toolbox.clone, offspring)

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        
        processes = 15   
        pool = MyPool(processes = processes, maxtasksperchild=1)    
        fitnesses = pool.map(toolbox.evaluate, invalid_ind)
        pool.terminate()
        pool.join()        

        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the parents + offspring
        pop[:] = elite + offspring
        
        print("Generation",g,"out of",NGEN,"COMPLETE")
        print(np.min([i.fitness.values[0] for i in pop]))
        
    

    return pop, pop[0], toolbox.evaluate(pop[0])

In [36]:
pop, pop0, sur_fit = GA(top, n=30, NGEN=15)

('Raw Z:', array([ 0.        ,  0.        ,  1.        ,  1.69263935, -1.80053191,
       -0.02216744,  0.        , -2.21495876,  2.25964051,  0.32257   ,
       -0.67075331,  4.23351949, 10.53752611, -0.74612903,  0.53694724,
       -1.55      , -1.46351611,  0.32257   , -0.67075331]), 'RT:', 0.11330294609069824)
('Raw Z:', array([ 0.        ,  0.        ,  1.        ,  1.69263935, -1.80053191,
       -0.02216744,  0.        , -2.21495876,  2.25964051,  0.32257   ,
       -0.67075331,  4.23351949, 10.53752611, -0.74612903,  0.53694724,
       -1.55      , -1.46351611,  0.32257   , -0.67075331]), 'RT:', 0.10534191131591797)('Raw Z:', array([ 0.        ,  0.        ,  6.        ,  1.69284873, -1.80053191,
       -0.02200866,  0.        , -2.21497355,  2.25949703,  0.34317799,
       -0.66192964,  4.98311339, 11.23914695, -0.74774194,  0.24144735,
       -2.175     , -1.48350918,  0.34317799, -0.66192964]), 'RT:', 0.11596488952636719)
('Raw Z:', array([ 0.        ,  0.        ,  1.      

        -1.55172414,   0.5288839 ,   2.85053381]), 'RT:', 27.35510516166687)
('Raw Z:', array([  0.        ,  -4.        , -40.        ,   1.63550068,
         2.80053191,  -0.85199067,   0.        ,   0.63698886,
         1.38584234,   0.5288839 ,   2.85053381,   6.        ,
        -4.625     ,  -0.75096774,   6.        ,   6.        ,
        -1.55172414,   0.5288839 ,   2.85053381]), 'RT:', 27.288766860961914)('Raw Z:', array([  0.        ,  -4.        , -40.        ,   2.423275  ,
         5.96542553,  -1.43359116,   0.        ,   1.75124591,
         2.03124212,   0.5288839 ,   2.85053381,   6.        ,
        -4.625     ,  -0.75096774,   6.        ,   6.        ,
        -1.55172414,   0.5288839 ,   2.85053381]), 'RT:', 27.276093006134033)
('Raw Z:', array([  0.        ,  -4.        , -40.        ,   0.86836353,
         2.77393617,  -0.63545428,   0.        ,   1.36911727,
         1.75032635,   0.5288839 ,   2.85053381,   6.        ,
        -4.625     ,  -0.75096774,   6.   


Traceback (most recent call last):
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/__init__.py", line 28, in generate_prediction
    result = self.generate_prediction_nocache(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 372, in generate_prediction_nocache
    return self.get_first_ap(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 367, in get_first_ap
    raise Exception("The voltage trace does not contain any detectable action potentials using method: " + self.threshold_method)
Exception: The voltage trace does not contain any detectable action potentials using method: dv/dt=20

('Raw Z:', array([  0.        ,  -4.        , -33.        ,   1.68347098,
         5.42021277,   0.45503989,   0.        ,   2.2015702 ,
         1.41965209,   0.54763121,  -0.69039146,   6.        ,
        -4.625     ,  -0.75096774,   6.        ,   6

        -1.53280422,   0.47488511,  -0.70749306]), 'RT:', 34.489760875701904)
('Generation', 0, 'out of', 15, 'COMPLETE')
12.09613583504997
Traceback (most recent call last):
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/__init__.py", line 28, in generate_prediction
    result = self.generate_prediction_nocache(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 372, in generate_prediction_nocache
    return self.get_first_ap(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 367, in get_first_ap
    raise Exception("The voltage trace does not contain any detectable action potentials using method: " + self.threshold_method)
Exception: The voltage trace does not contain any detectable action potentials using method: dv/dt=20

('Raw Z:', array([  0.        ,  -4.        , -40.        ,   1.1179895 ,
         2.81382979,  -1.65502411,   0.

Traceback (most recent call last):
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/__init__.py", line 28, in generate_prediction
    result = self.generate_prediction_nocache(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 372, in generate_prediction_nocache
    return self.get_first_ap(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 367, in get_first_ap
    raise Exception("The voltage trace does not contain any detectable action potentials using method: " + self.threshold_method)
Exception: The voltage trace does not contain any detectable action potentials using method: dv/dt=20


('Raw Z:', array([  0.        ,  -4.        , -40.        ,   0.74687473,
         2.56117021,  -1.80149134,   0.        ,   1.3521789 ,
         2.3530311 ,   0.5288839 ,   2.85053381,   6.        ,
        -4.625     ,  -0.75096774,   6.        ,   6

...

        -1.55172414,   0.5288839 ,   2.85053381]), 'RT:', 27.725041151046753)
Traceback (most recent call last):
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/__init__.py", line 28, in generate_prediction
    result = self.generate_prediction_nocache(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 372, in generate_prediction_nocache
    return self.get_first_ap(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 367, in get_first_ap
    raise Exception("The voltage trace does not contain any detectable action potentials using method: " + self.threshold_method)
Exception: The voltage trace does not contain any detectable action potentials using method: dv/dt=20

('Raw Z:', array([  0.        ,   4.        , -25.        ,  22.40427784,
         5.89893617,  -2.87078115,   0.        ,  -1.6889874 ,
         1.62273808,   1.84691157,  -0


('Raw Z:', array([  0.        ,   4.        , -14.        ,   7.46593142,
         7.6143617 ,  -2.18867367,   0.        ,  -0.02392646,
         2.2829558 ,   0.5396836 ,  -0.26913881,   6.        ,
        -4.625     ,  -0.75096774,   6.        ,   6.        ,
        -1.53842471,   0.5396836 ,  -0.26913881]), 'RT:', 27.75188398361206)
('Raw Z:', array([  0.        ,   0.        , -28.        ,   3.97432736,
        -1.68085106,  -0.50714221,   0.        ,  -2.2711772 ,
         3.32670635,   0.90307749,  -0.69039146,   5.18523998,
        11.61034288,  -0.74774194,   0.8715744 ,  -2.175     ,
         1.5024886 ,   0.90307749,  -0.69039146]), 'RT:', 27.28373694419861)
Traceback (most recent call last):
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/__init__.py", line 28, in generate_prediction
    result = self.generate_prediction_nocache(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line

        -1.55172414,   0.5288839 ,   2.85053381]), 'RT:', 26.66490888595581)
('Raw Z:', array([  0.        ,  -4.        , -40.        ,   1.01429579,
         2.69414894,  -0.86261959,   0.        ,   1.14168649,
         1.72337564,   0.5288839 ,   2.85053381,   6.        ,
        -4.625     ,  -0.75096774,   6.        ,   6.        ,
        -1.55172414,   0.5288839 ,   2.85053381]), 'RT:', 26.690202951431274)
Traceback (most recent call last):
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/__init__.py", line 28, in generate_prediction
    result = self.generate_prediction_nocache(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 372, in generate_prediction_nocache
    return self.get_first_ap(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 367, in get_first_ap
    raise Exception("The voltage trace does not contain any detectab


Traceback (most recent call last):
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/__init__.py", line 28, in generate_prediction
    result = self.generate_prediction_nocache(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 372, in generate_prediction_nocache
    return self.get_first_ap(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 367, in get_first_ap
    raise Exception("The voltage trace does not contain any detectable action potentials using method: " + self.threshold_method)
Exception: The voltage trace does not contain any detectable action potentials using method: dv/dt=20


('Raw Z:', array([  0.        ,  -4.        , -40.        ,   3.89755015,
         4.27659574,  -2.2456659 ,   0.        ,   0.07929898,
         1.99699034,   0.5288839 ,   2.85053381,   6.        ,
        -4.625     ,  -0.75096774,   6.        ,   


('Raw Z:', array([  0.        ,  -4.        , -32.        ,   1.61633925,
         4.91489362,   0.32217405,   0.        ,   1.96727445,
         1.48096557,   0.53643578,  -0.69039146,   6.        ,
        -4.625     ,  -0.75096774,   6.        ,   6.        ,
        -1.51536087,   0.53643578,  -0.69039146]), 'RT:', 27.83354616165161)
Traceback (most recent call last):
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/__init__.py", line 28, in generate_prediction
    result = self.generate_prediction_nocache(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 372, in generate_prediction_nocache
    return self.get_first_ap(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 367, in get_first_ap
    raise Exception("The voltage trace does not contain any detectable action potentials using method: " + self.threshold_method)
Exception: The 

In [39]:
for p in pop:
    print(p.fitness.values)

(12.09613583504997,)
(12.096709923146589,)
(12.098487309811935,)
(12.099936823247297,)
(12.102694420216903,)
(12.102932814954146,)
(45.113925673840804,)
(44.86010148388909,)
(45.09993268333857,)
(45.028163152151556,)
(44.97819944645228,)
(44.98923008514727,)
(45.28777518982302,)
(44.841022456916896,)
(44.922179598840835,)
(45.007825030762376,)
(45.15014386479507,)
(44.934009146368375,)
(45.114263448198514,)
(45.093078329112686,)
(24.02850623816832,)
(16.58659772714331,)
(44.100246806857825,)
(52.69711519691985,)
(45.29417577234999,)
(44.672985006121955,)
(44.85938243988705,)
(144.39330958324283,)
(31.86045218327875,)
(37.52116772026682,)
(23.522537552829448,)
(44.06009520340601,)
(16.779941435055893,)
(28.013902981794608,)
(33.465499805194874,)
(12.100174407361372,)
(43.86767621082261,)


In [38]:
import deap
top = deap.tools.selNSGA2(pop,8)
DataFrame([list(i) for i in top],[i.fitness for i in top])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
"(12.09613583504997,)",13.021558,0.142124,46.858416,-93.26653,-56.871313,0.000024,4.589941,12.492102,0.013737,0.012076,0.00101,0.001969,0.000646,0.000713,-29.718414,0.000012,0.003835
"(12.096709923146589,)",13.021558,0.142124,46.858416,-93.26653,-56.871313,0.000024,4.589941,12.492102,0.013737,0.012076,0.00101,0.001969,0.000646,0.000713,-29.718414,0.000012,0.003835
"(12.098487309811935,)",13.021558,0.142124,46.858416,-93.26653,-56.871313,0.000024,4.589941,12.492102,0.013737,0.012076,0.00101,0.001969,0.000646,0.000713,-29.718414,0.000012,0.003835
"(12.099936823247297,)",13.021558,0.142124,46.858416,-93.26653,-56.871313,0.000024,4.589941,12.492102,0.013737,0.012076,0.00101,0.001969,0.000646,0.000713,-29.718414,0.000012,0.003835
"(12.100174407361372,)",13.021558,0.142124,46.858416,-93.26653,-56.871313,0.000024,4.589941,12.492102,0.013737,0.012076,0.00101,0.001969,0.000646,0.000713,-29.718414,0.000012,0.003835
"(12.102694420216903,)",13.021558,0.142124,46.858416,-93.26653,-56.871313,0.000024,4.589941,12.492102,0.013737,0.012076,0.00101,0.001969,0.000646,0.000713,-29.718414,0.000012,0.003835
"(12.102932814954146,)",13.021558,0.142124,46.858416,-93.26653,-56.871313,0.000024,4.589941,12.492102,0.013737,0.012076,0.00101,0.001969,0.000646,0.000713,-29.718414,0.000012,0.003835
"(16.58659772714331,)",13.021558,0.153685,46.858416,-93.26653,-56.871313,0.000024,4.589941,12.492102,0.013737,0.012076,0.00101,0.001969,0.000646,0.000713,-39.922390,0.000012,0.003835


In [40]:
top[0]

[13.021558319332806,
 0.1421244817119445,
 46.8584157330876,
 -93.26652964252163,
 -56.87131277268493,
 2.4402946120574333e-05,
 4.58994085936241,
 12.492102024675049,
 0.0137369123653096,
 0.012076032916784003,
 0.001009786028927451,
 0.001969382122174609,
 0.0006458839218994715,
 0.0007132677073125959,
 -29.718414466812085,
 1.1532769221682631e-05,
 0.0038349927360426094]

In [ ]:
for i in range(10):
    genn, y_pred = retrain_surrogate()
    
    pop, pop0, sur_fit = GA(xt)

    # Select indivs 
    ga_pop = np.unique(pop,axis=0)

    new_xt, new_yt, new_grad_yt = get_true_points(ga_pop)
    xt =      np.append(xt,     new_xt,     axis=0)
    yt =      np.append(yt,     new_yt,     axis=0)
    dyt_dxt = np.append(dyt_dxt,new_grad_yt,axis=0)
    new_xt, new_yt, new_grad_yt
    
    best = np.argmin(yt)
    
    print("best fitness so far", yt[best])
    
    for pi, p in enumerate(params):
        print(p["attr"], xt[best][pi])

In [ ]:
xt[best]

In [ ]:
DataFrame(xt,yt).sort_index()

In [ ]:
xt.shape

In [ ]:
for c in range(xt.shape[1]):
    plt.plot(xt[:,c],yt,'o',label=params[c]["attr"])
    plt.show()